In [43]:
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import os
import pydicom
import time

from registration_utilities import *
from moviepy.editor import *

%matplotlib inline

# %%
def remove_keymap_conflicts(new_keys_set):
    for prop in plt.rcParams:
        if prop.startswith('keymap.'):
            keys = plt.rcParams[prop]
            remove_list = set(keys) & new_keys_set
            for key in remove_list:
                keys.remove(key)


def multi_slice_viewer(volume, mask=None):
    remove_keymap_conflicts({'j', 'k'})
    fig, ax = plt.subplots()

    if mask is not None:
        mask = mask * np.max(volume) / 2
        volume = volume + mask

    ax.volume = volume
    ax.index = volume.shape[2] // 2
    ax.imshow(volume[:, :, ax.index])

    fig.canvas.mpl_connect('key_press_event', process_key)


def process_key(event):
    fig = event.canvas.figure
    ax = fig.axes[0]
    if event.key == 'j':
        previous_slice(ax)
    elif event.key == 'k':
        next_slice(ax)
    fig.canvas.draw()


def previous_slice(ax):
    volume = ax.volume
    ax.index = (ax.index - 1) % volume.shape[2]
    ax.images[0].set_array(volume[:, :, ax.index])

def next_slice(ax):
    volume = ax.volume
    ax.index = (ax.index + 1) % volume.shape[2]
    ax.images[0].set_array(volume[:, :, ax.index])

#%%
PATIENTS_DIR = '/Users/riccardobusetti/Desktop/tmp_processed'

def gif(filename, array, fps=10, scale=1.0):
    """Creates a gif given a stack of images using moviepy
    Notes
    -----
    works with current Github version of moviepy (not the pip version)
    https://github.com/Zulko/moviepy/commit/d4c9c37bc88261d8ed8b5d9b7c317d13b2cdf62e
    Usage
    -----
    >>> X = randn(100, 64, 64)
    >>> gif('test.gif', X)
    Parameters
    ----------
    filename : string
        The filename of the gif to write to
    array : array_like
        A numpy array that contains a sequence of images
    fps : int
        frames per second (default: 10)
    scale : float
        how much to rescale each image by (default: 1.0)
    """

    # ensure that the file has the .gif extension
    fname, _ = os.path.splitext(filename)
    filename = fname + '.gif'

    # copy into the color dimension if the images are black and white
    if array.ndim == 3:
        array = array[..., np.newaxis] * np.ones(3)

    # make the moviepy clip
    clip = ImageSequenceClip(list(array), fps=fps).resize(scale)
    clip.write_gif(filename, fps=fps)
    return clip

def load_patients():
    return {
        patient_dir.name: {
            study_dir.name: {
                sequence_dir.name: sum(1 for _ in os.scandir(sequence_dir))
                for sequence_dir in os.scandir(study_dir) if not sequence_dir.name.startswith('.')
            }
            for study_dir in os.scandir(patient_dir) if not study_dir.name.startswith('.')
        }
        for patient_dir in os.scandir(PATIENTS_DIR) if not patient_dir.name.startswith('.')
    }


data = load_patients()

for patient in data:
    os.makedirs(f"/Users/riccardobusetti/Desktop/gifs/{patient}/")
    for study in data[patient]:
        os.makedirs(f"/Users/riccardobusetti/Desktop/gifs/{patient}/{study}")
        seqs = data[patient][study].items()
    
        for seq, _ in seqs:
            DIR = os.path.join(PATIENTS_DIR, patient, study, seq)
            
            reader_1 = sitk.ImageSeriesReader()
            reader_1.SetOutputPixelType(sitk.sitkFloat32)
            dicom_names_1 = reader_1.GetGDCMSeriesFileNames(DIR)
            reader_1.SetFileNames(dicom_names_1)
            
            try:
                image = reader_1.Execute()
            except RuntimeError:
                pass
            
            image_np = sitk.GetArrayFromImage(image)
            
            depth, height, width = image_np.shape
            
            gif(f"/Users/riccardobusetti/Desktop/gifs/{patient}/{study}/{patient}_{study}_{seq}.gif", image_np, 60)
            


[MoviePy] Building file /Users/riccardobusetti/Desktop/gifs/pa016/st000/pa016_st000_se003.gif with imageio


100%|██████████| 30/30 [00:00<00:00, 85.23it/s]



[MoviePy] Building file /Users/riccardobusetti/Desktop/gifs/pa016/st000/pa016_st000_se004.gif with imageio


100%|██████████| 41/41 [00:01<00:00, 36.12it/s]



[MoviePy] Building file /Users/riccardobusetti/Desktop/gifs/pa016/st000/pa016_st000_se005.gif with imageio


...

100%|██████████| 36/36 [00:00<00:00, 36.11it/s]



[MoviePy] Building file /Users/riccardobusetti/Desktop/gifs/pa001/st000/pa001_st000_se001.gif with imageio


100%|██████████| 192/192 [00:01<00:00, 106.36it/s]



[MoviePy] Building file /Users/riccardobusetti/Desktop/gifs/pa001/st000/pa001_st000_se006.gif with imageio


100%|██████████| 41/41 [00:00<00:00, 70.05it/s]
